In [1]:
# assignment 6
# MSDS 422
# Professor Anil Chaturvedi

# https://github.com/ageron/handson-ml/blob/master/10_introduction_to_artificial_neural_networks.ipynb
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
# https://github.com/ageron/handson-ml/blob/master/12_distributed_tensorflow.ipynb

# more info
# https://www.datacamp.com/community/tutorials/cnn-tensorflow-python
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

# really wonderful resource for noobs
# https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/

<div style="text-align: right"><b>pkg imports</b></div>

In [2]:
# basix
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import time

# prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

#modeling
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense

# other

%matplotlib inline   
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<div style="text-align: right"><b>configs, table setup</b></div>

In [3]:
"""
for sci-kit:
‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x
‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
‘relu’, the rectified linear unit function, returns f(x) = max(0, x)

for tf:
idk
"""

#sk_act_fun = "identity"
sk_act_fun = "logistic"
#sk_act_fun = "tanh"
#sk_act_fun = "relu"

tf_act_fun = "sigmoid"

In [4]:
pd.options.display.float_format = "{:,.4f}".format
RANDOM_SEED = 8675309
responseVar = "label"

In [5]:
# setup for consolidated table, later

table = {"model 1":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 2":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 3":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 4":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 5":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 6":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 7":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 8":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None}
        }

pd.DataFrame(table).transpose().head(2)

,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,None,None,None,None,None
model 2,None,None,None,None,None


<div style="text-align: right"><b>defining things for model</b></div>

In [6]:
# activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# lambda x: 1 / (1 + np.exp(-x))

In [7]:
training_iters = 10
learning_rate = 0.01
b_size = 128

# MNIST data input (img shape: 28*28)
n_input = 28

# MNIST total classes (0-9 digits)
n_classes = 10

<div style="text-align: right"><b>data setup for scikit</b></div>

In [8]:
sk_mnist_train = pd.read_csv('train.csv')
sk_mnist_test = pd.read_csv('test.csv')
#sk_mnist = pd.concat([mnist_test, mnist_train.drop(columns = "label")]) # combines the rows of 2 datasets

In [9]:
print("le shape of le train is le : ", sk_mnist_train.shape)
print("le shape of le test        : ", sk_mnist_test.shape)

le shape of le train is le :  (42000, 785)
le shape of le test        :  (28000, 784)


In [10]:
# 'test' data is our future data
sk_future_data = sk_mnist_test
sk_mnist = sk_mnist_train
del(sk_mnist_test)
del(sk_mnist_train)

In [11]:
try:
    sk_mnist_test
except NameError as e:
    print(e) # meow
    
try:
    sk_mnist_train
except NameError as e:
    print(e) # meow

name 'sk_mnist_test' is not defined
name 'sk_mnist_train' is not defined


In [12]:
#X = mnist_scaled.loc[:, mnist_scaled.columns != responseVar]
#y = mnist_scaled[responseVar]
sk_X = sk_mnist.loc[:, sk_mnist.columns != responseVar]
sk_y = sk_mnist[responseVar]

In [13]:
pixel744 = pd.DataFrame(sk_mnist.describe()["pixel774"]).loc[["count","mean","std","min","max"],]

In [14]:
scaler = StandardScaler()
sk_np_mnist_x = scaler.fit_transform(sk_X)
sk_X_scaled = pd.DataFrame(sk_np_mnist_x, index=sk_X.index, columns=sk_X.columns)

In [15]:
pd.DataFrame(sk_X_scaled.describe()["pixel774"]).loc[["count","mean","std","min","max"],]

,pixel774
count,"42,000.0000"
mean,0.0000
std,1.0000
min,-0.0347
max,40.2009


In [16]:
sk_X_train, sk_X_test, sk_y_train, sk_y_test = train_test_split(sk_X_scaled, sk_y,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=RANDOM_SEED)

<div style="text-align: right"><b>data setup for tensorflow</b></div>

In [17]:
from keras.datasets import mnist
(tf_X_train, tf_y_train), (tf_X_test, tf_y_test) = mnist.load_data()

In [18]:
tf_X_valid, tf_X_train = tf_X_train[:5000] / 255.0, tf_X_train[5000:] / 255.0
tf_y_valid, tf_y_train = tf_y_train[:5000], tf_y_train[5000:]
tf_X_test = tf_X_test / 255.0

<div style="text-align: right"><b>model 1 : 2x1. 2 layers w 1 neuron each. sci-kit</b></div>

In [19]:
print("approximate start time : ", time.asctime())
start1 = time.perf_counter()

approximate start time :  Sat Oct 24 13:15:17 2020


In [20]:
twox1 = MLPClassifier(hidden_layer_sizes = (2, 1), # neurons comma layers
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [21]:
stop1 = time.perf_counter()
tm1 = round(stop1 - start1)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm1)
print("time of execution in minutes : ", round(tm1/60, 1))

approximate completion time :  Sat Oct 24 13:16:08 2020
time of execution in seconds :  51
time of execution in minutes :  0.8


In [22]:
table["model 1"]["nbr_layers"] = "1"
table["model 1"]["nodes_p_layer"] = "2"
table["model 1"]["processing_time"] = tm1
table["model 1"]["testset_acc"] = twox1.score(sk_X_test, sk_y_test)
table["model 1"]["trainset_acc"] = twox1.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 2 : 2x2. 2 layers w 2 neurons each. I think. sci-kit</b></div>

In [23]:
print("approximate start time : ", time.asctime())
start2 = time.perf_counter()

approximate start time :  Sat Oct 24 13:16:08 2020


In [24]:
twox2 = MLPClassifier(hidden_layer_sizes = (2, 2), 
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [25]:
stop2 = time.perf_counter()
tm2 = round(stop2 - start2)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm2)
print("time of execution in minutes : ", round(tm2/60, 1))

approximate completion time :  Sat Oct 24 13:17:02 2020
time of execution in seconds :  54
time of execution in minutes :  0.9


In [27]:
table["model 2"]["nbr_layers"] = "2"
table["model 2"]["nodes_p_layer"] = "2"
table["model 2"]["processing_time"] = tm2
table["model 2"]["testset_acc"] = twox2.score(sk_X_test, sk_y_test)
table["model 2"]["trainset_acc"] = twox2.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 3 : x1. 30 layers w 4 neurons each. sci-kit</b></div>

In [28]:
print("approximate start time : ", time.asctime())
start3 = time.perf_counter()

approximate start time :  Sat Oct 24 13:17:53 2020


In [30]:
thirtyx4 = MLPClassifier(hidden_layer_sizes = (30, 4), 
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [31]:
stop3 = time.perf_counter()
tm3 = round(stop3 - start3)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm3)
print("time of execution in minutes : ", round(tm3/60, 1))

approximate completion time :  Sat Oct 24 13:20:01 2020
time of execution in seconds :  128
time of execution in minutes :  2.1


In [32]:
table["model 3"]["nbr_layers"] = "30"
table["model 3"]["nodes_p_layer"] = "4"
table["model 3"]["processing_time"] = tm3
table["model 3"]["testset_acc"] = thirtyx4.score(sk_X_test, sk_y_test)
table["model 3"]["trainset_acc"] = thirtyx4.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 4 : x1. 4 layers, each w 30 neurons. sci-kit</b></div>

In [33]:
print("approximate start time : ", time.asctime())
start4 = time.perf_counter()

approximate start time :  Sat Oct 24 13:20:02 2020


In [34]:
fourx30 = MLPClassifier(hidden_layer_sizes = (4, 30), 
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [35]:
stop4 = time.perf_counter()
tm4 = round(stop4 - start4)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm4)
print("time of execution in minutes : ", round(tm4/60, 1))

approximate completion time :  Sat Oct 24 13:21:13 2020
time of execution in seconds :  72
time of execution in minutes :  1.2


In [36]:
table["model 4"]["nbr_layers"] = "4"
table["model 4"]["nodes_p_layer"] = "30"
table["model 4"]["processing_time"] = tm4
table["model 4"]["testset_acc"] = fourx30.score(sk_X_test, sk_y_test)
table["model 4"]["trainset_acc"] = fourx30.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 5 : 2x1. 2 layers w 1 neurons each</b></div>

In [37]:
print("approximate start time : ", time.asctime())
start5 = time.perf_counter()

approximate start time :  Sat Oct 24 13:21:14 2020


In [38]:
#classification MLP with two hidden layers
model5 = Sequential()
model5.add(Flatten(input_shape=[28, 28]))
model5.add(Dense(300, activation=tf_act_fun))
model5.add(Dense(100, activation=tf_act_fun))
model5.add(Dense(10, activation="softmax"))

In [39]:
model5.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [40]:
model5.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [41]:
history5 = model5.fit(tf_X_train, tf_y_train, epochs=5,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/5
1719/1719 [==============================] - 4s 3ms/step - loss: 2.1608 - accuracy: 0.4014 - val_loss: 1.9402 - val_accuracy: 0.5986
Epoch 2/5
1719/1719 [==============================] - 5s 3ms/step - loss: 1.5732 - accuracy: 0.6628 - val_loss: 1.1943 - val_accuracy: 0.7200
Epoch 3/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.9874 - accuracy: 0.7666 - val_loss: 0.8069 - val_accuracy: 0.8116
Epoch 4/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7259 - accuracy: 0.8196 - val_loss: 0.6305 - val_accuracy: 0.8468
Epoch 5/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5944 - accuracy: 0.8479 - val_loss: 0.5315 - val_accuracy: 0.8658


In [42]:
stop5 = time.perf_counter()
tm5 = round(stop5 - start5)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm5)
print("time of execution in minutes : ", round(tm5/60, 1))

approximate completion time :  Sat Oct 24 13:21:38 2020
time of execution in seconds :  24
time of execution in minutes :  0.4


In [53]:
table["model 5"]["nbr_layers"] = "2"
table["model 5"]["nodes_p_layer"] = "1"
table["model 5"]["processing_time"] = tm5
table["model 5"]["testset_acc"] = model5.evaluate(tf_X_test, tf_y_test)[1]
table["model 5"]["trainset_acc"] = model5.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 3s 2ms/step - loss: 0.5398 - accuracy: 0.8611


<div style="text-align: right"><b>model 6 : 2x2. 2 layers w 2 neurons each</b></div>

In [58]:
print("approximate start time : ", time.asctime())
start6 = time.perf_counter()

approximate start time :  Sat Oct 24 13:23:07 2020


In [59]:
#classification MLP with two hidden layers
model6 = Sequential()
model6.add(Flatten(input_shape=[28, 28]))
model6.add(Dense(300, activation=tf_act_fun))
model6.add(Dense(100, activation=tf_act_fun))
model6.add(Dense(10, activation="softmax"))

In [60]:
model6.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [61]:
model6.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history6 = model5.fit(tf_X_train, tf_y_train, epochs=5,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5076 - accuracy: 0.8676 - val_loss: 0.4608 - val_accuracy: 0.8824
Epoch 2/5
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4577 - accuracy: 0.8773 - val_loss: 0.4186 - val_accuracy: 0.8902
Epoch 3/5
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4237 - accuracy: 0.8842 - val_loss: 0.3906 - val_accuracy: 0.8948
Epoch 4/5
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3995 - accuracy: 0.8897 - val_loss: 0.3703 - val_accuracy: 0.8978
Epoch 5/5
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3814 - accuracy: 0.8933 - val_loss: 0.3536 - val_accuracy: 0.9008


In [62]:
stop6 = time.perf_counter()
tm6 = round(stop6 - start6)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm6)
print("time of execution in minutes : ", round(tm6/60, 1))

approximate completion time :  Sat Oct 24 13:23:32 2020
time of execution in seconds :  24
time of execution in minutes :  0.4


In [63]:
table["model 6"]["nbr_layers"] = "2"
table["model 6"]["nodes_p_layer"] = "1"
table["model 6"]["processing_time"] = tm6
table["model 6"]["testset_acc"] = model6.evaluate(tf_X_test, tf_y_test)[1]
table["model 6"]["trainset_acc"] = model6.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 3s 2ms/step - loss: 2.4896 - accuracy: 0.1123


<div style="text-align: right"><b>model 7 : 30x4. 30 layers w 4 neurons each</b></div>

In [50]:
print("approximate start time : ", time.asctime())
start7 = time.perf_counter()

approximate start time :  Sat Oct 24 13:22:03 2020


<div style="text-align: right"><b>model 8 : 4x30. 4 layers w 30 neurons each</b></div>

In [51]:
print("approximate start time : ", time.asctime())
start8 = time.perf_counter()

approximate start time :  Sat Oct 24 13:22:03 2020


In [64]:
print("processing time in seconds : ")
pd.DataFrame(table).transpose()

processing time in seconds : 


,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,1,2,51,0.4338,0.4290
model 2,2,2,54,0.6693,0.6603
model 3,30,4,128,0.9971,0.9025
model 4,4,30,72,0.8773,0.8425
model 5,2,1,49,0.8611,0.8652
model 6,2,1,24,0.1123,0.1135
model 7,None,None,None,None,None
model 8,None,None,None,None,None
